<a href="https://colab.research.google.com/github/jackylmw/Week-2_Data_Preprocessing/blob/Assignment2_JackyLam/JackyLam_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import drive for subsequent applications

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Required libraries

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Merge EmoDB and RAVDESS

The actor's ID in EmoDB dataset will be represented using the representation in the RAVDESS dataset

In [3]:
actor_id = {
    "03": "25",
    "08": "26",
    "10": "27",
    "09": "28",
    "11": "29",
    "13": "30",
    "12": "31",
    "14": "32",
    "15": "33",
    "16": "34",
}

Since the emotion categories in the EmoDB dataset are not exactly consistent with those in the RAVDESS dataset, I think bored in EmoDB is calm in RAVDESS. Since the EmoDB does not contain the emotion of surprised, I left it alone, which meant keeping the emotion number of surprised in the original RAVDESS dataset.

In [4]:
emotion = {"W": "05",
           "E": "07",
           "A": "06",
           "F": "03",
           "T": "04",
           "N": "01",
           "L": "02"}

In the EmoDB dataset, different versions are represented by letters, while in RAVDESS, different versions are represented by numbers. To make the EmoDB consistent with RAVDESS naming, I will convert letters to numbers.

In [5]:
versions = {"a": "01", "b": "02", "c": "03", "d": "04", "e": "05", "f": "06"}

In [8]:
import os

os.chdir(os.path.dirname("/content/drive/MyDrive/IAT481-Assignment2/wav"))

if not os.path.exists("./RAVDESS"):
    os.mkdir("./RAVDESS")

files = os.listdir("./wav")

for f in files:
    new_f = (
        "03-01-"
        + emotion[f[5]]
        + "-01-"
        + f[2:5]
        + "-"
        + versions[f[6]]
        + "-"
        + actor_id[f[0:2]]
        + ".wav"
    )

    if not os.path.exists("./RAVDESS/Actor_" + actor_id[f[0:2]]):
        os.mkdir("./RAVDESS/Actor_" + actor_id[f[0:2]])


    os.rename("./wav/" + f, "./RAVDESS/Actor_" + actor_id[f[0:2]] + "/" + new_f)